###### 겨리 - 2020/12/22
### **LU분해(LU Decomposition)**
#### 해집합을 구하는 효율적인 방법
##### - L(Lower triangular matrix) + U(Upper triangular matrix)
##### - 단위행렬(I)은 상삼각행렬이면서 하삼각행렬
###### &nbsp;&nbsp; 삼각행렬을 사용하면 Back/Forward Substitution 과정이 더 효율적(O(n^2), 일반적인 가우스 조르당 소거법의 경우 O(n^3))
##### - 행렬 A를 L(기본행 연산들의 역행렬)과 U(행사다리꼴)의 행렬곱으로 분해
###### &nbsp;&nbsp; * LU 분해의 결과는 유일하지 않음
##### - LU분해를 전처리 후 방정식을 분산처리할 수 있는 장점이 있음

In [46]:
import numpy as np
import random

#### 기본 성질
##### 1. 가우스 소거법 연산에 사용되는 기본행렬(E)은 모두 하삼각행렬이다. #행교환 연산은 제외
##### 2. 1의 기본행렬(=하삼각행렬)의 역행렬은 하삼각행렬이다. 
##### 3. 하삼각행렬끼리의 행렬곱 연산 결과는 하삼각행렬이다.
### 
#### 전개(컴퓨터 연산)
##### 1. 행렬 A를 가우스 소거법을 사용하여 행사다리꼴(U)로 분해한다(이때, 다음 계산을 용이하게 하기 위해 U의 선행 원소들을 1로 만들어준다).
##### 2. 분해에 사용된 기본행 연산의 역행렬을 기록하여 행렬 L로 정의한다. 
##### -> 만약 첫 원소가 0일 경우 행교환을 통해 변환(=PLU분해)

In [47]:
def lu_decomposition(matrix):
    
    if len(matrix) == len(matrix[0]):
        dim = len(matrix)
        L = np.zeros((dim,dim))
    else : 
        print("정방행렬이 아닙니다.")


    #첫 원소가 0일 경우 행교환이 필요
    while matrix[0][0] == 0:
        temp = matrix[0].copy()
        matrix[0] = matrix[1].copy()
        matrix[1] = temp.copy()
    

    U=matrix.copy()
    #입력 행렬의 1차 검증 : 값이 모두 0인 열이 있는지 -> 변수의 값을 구할 수 없음
    idx = 0
    for c in range(dim): #열
        for r in range(dim): #행
            idx += U[r][c]

        if idx == 0: #첫 열이 모두 0
            print(c+1,"열 의 값이 모두 0입니다.")

            
    for i in range(dim):
        ero_product = U[i][i].copy() # Elementary Row Operation(기본행연산)
        if ero_product == 0:
            pass
        else :
            L[i][i] = ero_product
            U[i] /= ero_product
            for j in range(i+1,dim):
                ero_sum = U[j][i].copy() 
                L[j][i] = ero_sum
                U[j] -= ero_sum*U[i].copy()
    
    #검증(L,U의 곱이 입력한 행렬과 같은지)
    if matrix.all() == np.dot(L,U).all():
        print("분해 완료")
    else : 
        print("분해 실패")
        return 
    

    return L,U

In [48]:
A=np.array([[2.,2.,1.],[2.,1.,3.],[1.,2.,3.]])
b=np.array([[11.,11.,10]]) #결과

In [49]:
L,U= lu_decomposition(A)

분해 완료


In [50]:
L #하삼각행렬 = 기본행 연산들의 행렬곱의 역행렬

array([[ 2. ,  0. ,  0. ],
       [ 2. , -1. ,  0. ],
       [ 1. ,  1. ,  4.5]])

In [51]:
U #상삼각행렬 = Back Substitution 이전의 행사다리꼴(해 제외)

array([[ 1. ,  1. ,  0.5],
       [-0. ,  1. , -2. ],
       [ 0. ,  0. ,  1. ]])

In [52]:
np.dot(L,U) #L,U를 곱한 값은(첫 원소가 0일 경우에는 행교환 후의) 행렬 A와 같음

array([[2., 2., 1.],
       [2., 1., 3.],
       [1., 2., 3.]])

#### LU분해를 활용한 방정식(Ax=b)의 해 구하기
##### 첨가행렬(augmented matrix) 정의

In [53]:
Lb=np.concatenate((L,b.T),axis=1) #Lz=b의 첨가행렬
np.around(Lb,2)

array([[ 2. ,  0. ,  0. , 11. ],
       [ 2. , -1. ,  0. , 11. ],
       [ 1. ,  1. ,  4.5, 10. ]])

#### Ax=b -> LUx=b
##### 이때  Ux를 z(3x1 행렬)로 정의하면 Lz = b
##### -> 방정식의 해(z)를 구하기 위해서는 L(하부삼각)을 Forward Substitution(가우스-조르당 소거법의 반대 연산)하여야 함

In [54]:
def forward_substitution(aug_matrix):
    mcol = len(aug_matrix[0])
    for col in range(mcol-1):
        for row in range(col+1,mcol-1): #col/row를 파이썬 인덱스에 맞게 적용
            const = aug_matrix[row][col]/aug_matrix[col][col] * -1  #constraint
            print(col+1,"행을",round(const,2), "만큼 상수배하여", row+1, "행의", col+1,"열을 소거")
            aug_matrix[row] += const * aug_matrix[col]
            print(np.around(aug_matrix,2)) #출력을 위한 반올림
            print()
    
    for i in range(mcol-1): #행의 갯수만큼
        aug_matrix[i] /= aug_matrix[i][i] # 각 행의 선행원소를 1로 만들어 줌
            
    
    print("선행원소를 1로 만드는 연산")
    print(aug_matrix)
    
    return aug_matrix[:,-1].reshape(1,-1) #실제 해는 마지막 열(앞은 계수행렬 = 단위행렬), 열벡터 형태로 반환

In [55]:
z= forward_substitution(Lb)
z

1 행을 -1.0 만큼 상수배하여 2 행의 1 열을 소거
[[ 2.   0.   0.  11. ]
 [ 0.  -1.   0.   0. ]
 [ 1.   1.   4.5 10. ]]

1 행을 -0.5 만큼 상수배하여 3 행의 1 열을 소거
[[ 2.   0.   0.  11. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   1.   4.5  4.5]]

2 행을 1.0 만큼 상수배하여 3 행의 2 열을 소거
[[ 2.   0.   0.  11. ]
 [ 0.  -1.   0.   0. ]
 [ 0.   0.   4.5  4.5]]

선행원소를 1로 만드는 연산
[[ 1.   0.   0.   5.5]
 [-0.   1.  -0.  -0. ]
 [ 0.   0.   1.   1. ]]


array([[ 5.5, -0. ,  1. ]])

#### z의 해가 구해졌으므로, Ux = z의 해인 x를 구할 수 있음
##### U는 상부삼각행렬이므로, U와 z를 첨가행렬로 구성하여 Back Substitution 사용
##### Back Substitution의 알고리즘은 반복되는 행/열의 순서가 반대인 것을 제외하고는 Forward와 동일함

In [56]:
def back_substitution(aug_matrix):
    mcol = len(aug_matrix[0])
    for col in range(mcol-2,0,-1):
        for row in range(col-1,-1,-1): #col/row를 파이썬 인덱스에 맞게 적용
            const = aug_matrix[row][col]/aug_matrix[col][col] * -1 #
            print(col+1,"행을", round(const,2), "만큼 상수배하여", row+1, "행의", col+1,"열을 소거")
            aug_matrix[row] += const * aug_matrix[col]
            print(np.around(aug_matrix,2)) #출력을 위한 반올림
    
    
    for i in range(mcol-1): #행의 갯수만큼
        aug_matrix[i] /= aug_matrix[i][i] # 각 행의 선행원소를 1로 만들어 줌
    
    return aug_matrix

In [57]:
U

array([[ 1. ,  1. ,  0.5],
       [-0. ,  1. , -2. ],
       [ 0. ,  0. ,  1. ]])

In [58]:
Uz=np.concatenate((U,z.T),axis=1) #Lz=b의 첨가행렬
np.around(Uz,4)

array([[ 1. ,  1. ,  0.5,  5.5],
       [-0. ,  1. , -2. , -0. ],
       [ 0. ,  0. ,  1. ,  1. ]])

In [59]:
x=back_substitution(Uz)

3 행을 2.0 만큼 상수배하여 2 행의 3 열을 소거
[[1.  1.  0.5 5.5]
 [0.  1.  0.  2. ]
 [0.  0.  1.  1. ]]
3 행을 -0.5 만큼 상수배하여 1 행의 3 열을 소거
[[1. 1. 0. 5.]
 [0. 1. 0. 2.]
 [0. 0. 1. 1.]]
2 행을 -1.0 만큼 상수배하여 1 행의 2 열을 소거
[[1. 0. 0. 3.]
 [0. 1. 0. 2.]
 [0. 0. 1. 1.]]


In [60]:
x[:,-1] # 방정식의 해

array([3., 2., 1.])

#### 가우스-조르당 소거법 알고리즘을 사용해도 동일한 해가 반환됨

In [43]:
Ab =np.concatenate((A,b.T),axis=1) #Ax=b의 첨가행렬
mat = gauss_jordan_elimination(Ab)
mat[:,-1]

array([3., 2., 1.])

# 
# 
# 
# 
# 
#### 다른 노트에서 정의한 함수

In [21]:
def random_matrix(rows,cols, max_no=10):
    row_list=[]
    for r in range(rows):
        col_list=[]
        for c in range(cols):
            col_list.append(float(random.randint(0,max_no))) 
        row_list.append(col_list)
        
    return np.array(row_list) 

def gauss_jordan_elimination(matrix):
    mcol = len(matrix[0])
    mrow = len(matrix)
    row_count = 0


    idx = 0
    for c in range(mcol): 
        for r in range(mrow):
            idx += matrix[r][c]

        if idx == 0: 
            print(c+1,"열 의 값이 모두 0입니다.")
            return 
            
        if mrow != mcol - 1:
            print("해가 무수히 많거나 없습니다.")
            return 

    for col_count in range(mcol-1):
        comp_num = np.inf 
        

        for i in range(row_count,mrow):
            if (0 < abs(matrix[i][col_count])) & (abs(matrix[i][col_count]) < abs(comp_num)) : 
                comp_num = matrix[i][col_count] 

                first_row = i
                if matrix[i][0] == 1:
                    break


        matrix[first_row]=matrix[first_row]/comp_num 
        matrix[first_row], matrix[row_count] = matrix[row_count].copy(), matrix[first_row].copy() 


        for j in range(row_count+1,mrow):
            if matrix[j][col_count] != 0:
                con_no = matrix[j][col_count]
                matrix[j] = matrix[j] - con_no * matrix[row_count]

        row_count += 1


    for col in range(mcol-2,0,-1):
        for row in range(col-1,-1,-1):
            const = matrix[row][col] * -1 
            matrix[row] += const * matrix[col]
    return  matrix